<a href="https://colab.research.google.com/github/batmanvane/complex-systems-modeling/blob/main/notebooks/06_phase_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# First, run this cell to enable widgets in Colab:
# !pip install ipywidgets
# from google.colab import output
# output.enable_custom_widget_manager()

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from ipywidgets import interactive, FloatSlider, VBox, HBox, Label
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Define the double-well potential
def potential(x, a=1, b=1):
    """Double-well potential: V(x) = -a*x^2/2 + b*x^4/4"""
    return -a * x**2 / 2 + b * x**4 / 4

def force(x, a=1, b=1):
    """Force: F = -dV/dx = a*x - b*x^3"""
    return a * x - b * x**3

# Equations of motion with damping
def equations(state, t, a, b, gamma):
    """
    state = [x, v] where x is position, v is velocity
    dx/dt = v
    dv/dt = -gamma*v + F(x) = -gamma*v + a*x - b*x^3
    """
    x, v = state
    dxdt = v
    dvdt = -gamma * v + force(x, a, b)
    return [dxdt, dvdt]

# Calculate basin of attraction
def basin_of_attraction(x_range, v_range, a, b, gamma, t_max=50):
    """Calculate which attractor each initial condition converges to"""
    basin = np.zeros((len(v_range), len(x_range)))

    # Find equilibrium points (attractors)
    if a/b > 0:
        x_eq = np.array([-np.sqrt(a/b), 0, np.sqrt(a/b)])
    else:
        x_eq = np.array([0])

    for i, v0 in enumerate(v_range):
        for j, x0 in enumerate(x_range):
            # Integrate trajectory
            t = np.linspace(0, t_max, 1000)
            try:
                sol = odeint(equations, [x0, v0], t, args=(a, b, gamma))
                x_final = sol[-1, 0]
                distances = np.abs(x_eq - x_final)
                basin[i, j] = np.argmin(distances)
            except:
                basin[i, j] = 0

    return basin

def plot_phase_space(a, b, gamma, x0, v0):
    """Create all plots with current parameters"""

    # Create figure
    fig = plt.figure(figsize=(18, 10))

    # Create subplots
    ax1 = plt.subplot(2, 3, 1)  # Potential
    ax2 = plt.subplot(2, 3, 2)  # Phase space with trajectory
    ax3 = plt.subplot(2, 3, 3)  # Basin of attraction
    ax4 = plt.subplot(2, 3, (4, 6))  # Time evolution

    # Plot 1: Potential
    x_pot = np.linspace(-2, 2, 300)
    V = potential(x_pot, a, b)
    ax1.plot(x_pot, V, 'b-', linewidth=2)
    ax1.set_xlabel('Position x', fontsize=12)
    ax1.set_ylabel('Energy', fontsize=12)
    ax1.set_title('Double-Well Potential', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)

    # Mark equilibrium points
    if a/b > 0:
        x_eq = np.array([-np.sqrt(a/b), 0, np.sqrt(a/b)])
        V_eq = potential(x_eq, a, b)
        ax1.plot(x_eq[[0, 2]], V_eq[[0, 2]], 'go', markersize=12, label='Stable', zorder=5)
        ax1.plot(x_eq[1], V_eq[1], 'ro', markersize=12, label='Unstable', zorder=5)
        ax1.legend(fontsize=10)

    # Plot 2: Phase space trajectory
    t = np.linspace(0, 30, 2000)
    sol = odeint(equations, [x0, v0], t, args=(a, b, gamma))

    ax2.plot(sol[:, 0], sol[:, 1], 'b-', alpha=0.6, linewidth=2)
    ax2.plot(x0, v0, 'go', markersize=12, label='Start', zorder=5)
    ax2.plot(sol[-1, 0], sol[-1, 1], 'rs', markersize=12, label='End', zorder=5)

    # Plot vector field
    x_field = np.linspace(-2, 2, 20)
    v_field = np.linspace(-2, 2, 20)
    X, V_grid = np.meshgrid(x_field, v_field)
    dX = V_grid
    dV = -gamma * V_grid + force(X, a, b)

    # Normalize vectors for better visualization
    M = np.sqrt(dX**2 + dV**2)
    M[M == 0] = 1
    dX_norm = dX / M
    dV_norm = dV / M

    ax2.quiver(X, V_grid, dX_norm, dV_norm, M, alpha=0.4, cmap='viridis', scale=30)

    # Mark attractors
    if a/b > 0:
        ax2.plot(x_eq[[0, 2]], [0, 0], 'go', markersize=14, label='Attractors', zorder=5)
        ax2.plot(x_eq[1], 0, 'ro', markersize=14, label='Saddle', zorder=5)

    ax2.set_xlabel('Position x', fontsize=12)
    ax2.set_ylabel('Velocity v', fontsize=12)
    ax2.set_title('Phase Space', fontsize=13, fontweight='bold')
    ax2.set_xlim(-2, 2)
    ax2.set_ylim(-2, 2)
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper right', fontsize=10)

    # Plot 3: Basin of attraction
    x_basin = np.linspace(-2, 2, 60)
    v_basin = np.linspace(-2, 2, 60)
    basin = basin_of_attraction(x_basin, v_basin, a, b, gamma, t_max=30)

    extent = [x_basin[0], x_basin[-1], v_basin[0], v_basin[-1]]
    im = ax3.imshow(basin, extent=extent, origin='lower', cmap='RdYlBu',
                    aspect='auto', alpha=0.7, interpolation='bilinear')

    ax3.plot(x0, v0, 'ko', markersize=12, markeredgewidth=3,
             markerfacecolor='lime', label='Initial condition', zorder=5)

    if a/b > 0:
        ax3.plot(x_eq[[0, 2]], [0, 0], 'go', markersize=14, label='Attractors', zorder=5)
        ax3.plot(x_eq[1], 0, 'ro', markersize=14, label='Saddle', zorder=5)

    ax3.set_xlabel('Position x', fontsize=12)
    ax3.set_ylabel('Velocity v', fontsize=12)
    ax3.set_title('Basin of Attraction', fontsize=13, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3, color='white', linewidth=0.5)

    # Plot 4: Time evolution
    ax4.plot(t, sol[:, 0], 'b-', linewidth=2.5, label='Position x(t)')
    ax4.plot(t, sol[:, 1], 'r-', linewidth=2.5, label='Velocity v(t)')
    ax4.set_xlabel('Time t', fontsize=12)
    ax4.set_ylabel('State', fontsize=12)
    ax4.set_title('Time Evolution', fontsize=13, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.legend(fontsize=11)
    ax4.axhline(y=0, color='k', linestyle='--', alpha=0.3)

    plt.suptitle('Interactive Double-Well Potential Phase Space Explorer',
                 fontsize=15, fontweight='bold', y=0.98)

    plt.tight_layout()
    plt.show()

# Print instructions
print("="*70)
print("DOUBLE-WELL POTENTIAL PHASE SPACE EXPLORER")
print("="*70)
print("\nInstructions:")
print("- Adjust sliders below to change system parameters")
print("- a: controls depth of potential wells")
print("- b: controls shape/width of potential wells")
print("- γ (gamma): damping coefficient")
print("- x₀, v₀: initial position and velocity")
print("\nFeatures:")
print("- Top left: Potential energy landscape")
print("- Top middle: Phase space with trajectory and vector field")
print("- Top right: Basin of attraction (which attractor you end up at)")
print("- Bottom: Time evolution of position and velocity")
print("\nTry different initial conditions to jump between attractors!")
print("="*70)
print()

# Create sliders
slider_a = FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0,
                       description='a (well depth):', continuous_update=False)
slider_b = FloatSlider(min=0.1, max=3.0, step=0.1, value=1.0,
                       description='b (well shape):', continuous_update=False)
slider_gamma = FloatSlider(min=0.0, max=1.0, step=0.05, value=0.3,
                           description='γ (damping):', continuous_update=False)
slider_x0 = FloatSlider(min=-2.0, max=2.0, step=0.1, value=-1.5,
                        description='x₀ (init pos):', continuous_update=False)
slider_v0 = FloatSlider(min=-2.0, max=2.0, step=0.1, value=0.0,
                        description='v₀ (init vel):', continuous_update=False)

# Create interactive widget
w = interactive(plot_phase_space,
                a=slider_a,
                b=slider_b,
                gamma=slider_gamma,
                x0=slider_x0,
                v0=slider_v0)

display(w)

DOUBLE-WELL POTENTIAL PHASE SPACE EXPLORER

Instructions:
- Adjust sliders below to change system parameters
- a: controls depth of potential wells
- b: controls shape/width of potential wells
- γ (gamma): damping coefficient
- x₀, v₀: initial position and velocity

Features:
- Top left: Potential energy landscape
- Top middle: Phase space with trajectory and vector field
- Top right: Basin of attraction (which attractor you end up at)
- Bottom: Time evolution of position and velocity

Try different initial conditions to jump between attractors!



interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='a (well depth):', max=3.0, …